# Objetivos:
 - implementação dos seguintes metodos de algoritmos de metodos de alcançar os zeros reais de uma função:
   - Método da Bisseção
   - Método da Falsa Posição
   - Método do Ponto Fixo
   - Método de Newton-Raphson
   - Método da Secante

In [34]:
import math
import time

#! Funções requeridas
def funcao_1(x): return math.e**(-x**2) - math.cos(x)

def funcao_2(x): return x**3 - x - 1

def funcao_3(x): return 4*math.sin(x) - math.e**(x)

def funcao_4(x): return x*math.log(x) - 1

epsolon = [
    10**(-4),
    10**(-6),
    10**(-5),
    10**(-7),
]

interval = [
    (1,2),
    (0,1),
    (2,3), 
]

def apply_all_functions(func, inter_max):
    results = []
    results.append(func(funcao_1, interval[0][0], interval[0][1], epsolon[0], inter_max))
    results.append(func(funcao_2, interval[1][0], interval[1][1], epsolon[1], inter_max))
    results.append(func(funcao_3, interval[2][0], interval[2][1], epsolon[2], inter_max))
    results.append(func(funcao_4, interval[0][0], interval[0][1], epsolon[3], inter_max))
    return results

In [35]:
# Teste se os intervalos são válidos
print("Verificação de intervalos:")
print(f"f1(1) * f1(2) = {funcao_1(1) * funcao_1(2):.6f} {'✓ OK' if funcao_1(1) * funcao_1(2) < 0 else '✗ INVÁLIDO'}")
print(f"f2(0) * f2(1) = {funcao_2(0) * funcao_2(1):.6f} {'✓ OK' if funcao_2(0) * funcao_2(1) < 0 else '✗ INVÁLIDO'}")
print(f"f3(2) * f3(3) = {funcao_3(2) * funcao_3(3):.6f} {'✓ OK' if funcao_3(2) * funcao_3(3) < 0 else '✗ INVÁLIDO'}")
print(f"f4(1) * f4(2) = {funcao_4(1) * funcao_4(2):.6f} {'✓ OK' if funcao_4(1) * funcao_4(2) < 0 else '✗ INVÁLIDO'}")

Verificação de intervalos:
f1(1) * f1(2) = -0.074911 ✓ OK
f2(0) * f2(1) = 1.000000 ✗ INVÁLIDO
f3(2) * f3(3) = 73.240397 ✗ INVÁLIDO
f4(1) * f4(2) = -0.386294 ✓ OK


## Metodo da Bisseção
O objetivo desse metodo é reduzir a amplitude do intervalo que contenha a raiz até se atingir a presisão requerida: (b-a) < ε. Iterando sussecivas vezes subistituindo o valor de a ou b pelo valor do ponto médio c = (a+b)/2, dependendo do sinal da função f(c).

In [36]:

def bissecao(f, a, b, precision, inter_max) -> tuple[float | None, int, int, float, float]:
    """
    f: Descrição
    a: Descrição
    b: Descrição
    precision: Descrição
    inter_max: Descrição
    """
    # Variaveis de controle
    inicio = time.time()
    count = 0
    f_count = 0
    iteract_timeList = []
    raiz = None

    # Verifica se o intervalo é valido
    fa = f(a)
    fb = f(b)
    f_count += 2
    
    if fa * fb > 0: return raiz, count, f_count, (time.time() - inicio), 0                                                        # Intervalo invalido
    count = 1
    
    while count <= inter_max:
        iteract_time = time.time()
        x = (a + b) / 2                                                                                                           # Ponto medio do intervalo [a, b]
        fx = f(x)                                                                                                                 # Avaliacao da funcao no ponto medio
        f_count += 1

        if abs(fx) < precision or (b - a) / 2 < precision:                                                                        #Verifica se a precisao foi atingida
            raiz =  x
            break

        if fa * fx > 0:                                                                                                           # Se o zero esta no intervalo [x, b] >> atualiza a = x
            a = x
            fa = fx
        else:                                                                                                                     # Se o zero esta no intervalo [a, x] >> atualiza b = x
            b = x
        
        count += 1
        iteract_timeList.append(time.time() - iteract_time)
    
    if raiz is None:
        raiz = (a + b) / 2

    iteract_time_avg = sum(iteract_timeList) / len(iteract_timeList) if iteract_timeList else 0

    return raiz, count, f_count, (time.time() - inicio), iteract_time_avg



raiz_1 = apply_all_functions(bissecao, 100)

funcoes = ['Função 1: e^(-x²) - cos(x)', 
           'Função 2: x³ - x - 1', 
           'Função 3: 4sin(x) - e^x', 
           'Função 4: x*log(x) - 1']

for idx, resultado in enumerate(raiz_1):
    raiz, iteracoes, avaliacoes, tempo_total, tempo_medio = resultado
    
    print(f"\n{funcoes[idx]}:")
    if raiz is not None:
        print(f"  Raiz encontrada: {raiz:.10f}")
        print(f"  Iterações: {iteracoes}")
        print(f"  Avaliações de f(x): {avaliacoes}")
        print(f"  Tempo total: {tempo_total:.6f}s")
        print(f"  Tempo médio por iteração: {tempo_medio:.9f}s")
    else:
        print("  ERRO: Intervalo inválido (f(a) * f(b) > 0)")


Função 1: e^(-x²) - cos(x):
  Raiz encontrada: 1.4472656250
  Iterações: 9
  Avaliações de f(x): 11
  Tempo total: 0.000016s
  Tempo médio por iteração: 0.000001013s

Função 2: x³ - x - 1:
  ERRO: Intervalo inválido (f(a) * f(b) > 0)

Função 3: 4sin(x) - e^x:
  ERRO: Intervalo inválido (f(a) * f(b) > 0)

Função 4: x*log(x) - 1:
  Raiz encontrada: 1.7632228136
  Iterações: 23
  Avaliações de f(x): 25
  Tempo total: 0.000011s
  Tempo médio por iteração: 0.000000271s


### Análise de Desempenho
* **Convergência:** Linear e garantida (desde que o intervalo inicial seja válido).
* **Custo Computacional:** Baixo custo por iteração (operações aritméticas simples), mas requer um número elevado de iterações para atingir alta precisão.

## Metodo da posição falsa
No caso da bisceção, x é o ponto medio aritmetico entre a e b. No metodo da posiçao falsa, x é o ponto onde a reta que liga os pontos (a, f(a)) e (b, f(b)) intercepta o eixo x.
ou seja:
$$
x = (\frac{aF(b) - bF(a)}{F(b) - F(a)}) 
$$


In [37]:
def falsa_posicao(f, a, b, precision, inter_max):
    """
    f: Descrição
    a: Descrição
    b: Descrição
    precision: Descrição
    inter_max: Descrição
    """
    inicio = time.time()
    count = 0
    f_count = 0
    iteract_timeList = []
    raiz = None

    fa = f(a)
    fb = f(b)
    f_count += 2
    
    if fa * fb > 0: return raiz, count, f_count, (time.time() - inicio), 0                          # Intervalo invalido
    count = 1

    x = (a*fb - b*fa) / (fb - fa)                                                                   # Ponto de interseccao da reta com o eixo x

    while count <= inter_max:
        iteract_time = time.time()
        x = (a*fb - b*fa) / (fb - fa)                                                               # Ponto de interseccao da reta com o eixo x
        fx = f(x)
        f_count += 1

        if abs(fx) < precision or abs(b - a) < precision: 
            raiz = x
            break

        if fa * fx > 0:                                                                             # Se o zero esta no intervalo [x, b] >> atualiza a = x
            a = x
            fa = fx
        else:                                                                                       # Se o zero esta no intervalo [a, x] >> atualiza b = x
            b = x
            fb = fx
        
        count += 1
        iteract_timeList.append(time.time() - iteract_time)
    
    if raiz is None:
        raiz = x
        
    iteract_time_avg = sum(iteract_timeList) / len(iteract_timeList) if iteract_timeList else 0
    return raiz, count, f_count, (time.time() - inicio), iteract_time_avg

raiz_2 = apply_all_functions(falsa_posicao, 100)

for i in raiz_2:
    print(i)

(1.4473570678005705, 6, 8, 1.52587890625e-05, 1.33514404296875e-06)
(None, 0, 2, 9.5367431640625e-07, 0)
(None, 0, 2, 1.430511474609375e-06, 0)
(1.7632228302998445, 6, 8, 5.0067901611328125e-06, 4.76837158203125e-07)


### Análise de Desempenho
* **Convergência:** Geralmente **superlinear** e mais rápida que a Bissecção para funções bem comportadas, pois "pula" etapas ao usar a inclinação da reta.
* **Ponto Crítico (Extremidade Fixa):** Se a função for estritamente côncava ou convexa no intervalo, uma das extremidades pode permanecer fixa durante todo o processo. Isso pode degradar a convergência para um comportamento linear, tornando-o, em casos raros, mais lento que a Bissecção.
* **Custo Computacional:** Similar à Bissecção (1 avaliação de função por iteração), com um leve acréscimo de operações aritméticas na fórmula da secante.

## Metodo do Ponto Fixo
O MPF consiste em transformar a equação f(x) = 0 em x = phi(x) e apartir de uma aproximação inicial $x_0$ phiera a sequencia {$x_k$} de aproximações para ɛ pela relação x_k+1 = phi(x_k), pois a função phi é tal que f(ɛ) = 0 se e somente se phi(ɛ) = ɛ. Transformando, assim o problema de encontrar um zero de f(x) mo problema de encontrar um ponto fixo de phi(x).
Diferente dos métodos de quebra de intervalo, o MPF parte de um chute inicial $x_0$ e gera uma sequência de aproximações $x_1, x_2, \dots, x_k$ através da relação de recorrência:
$$x_{k+1} = \phi(x_k)$$

In [38]:
# F1: e^(-x^2) - cos(x) = 0  =>  x = arccos(e^(-x^2))
def phi_1(x): 
    try:
        val = math.exp(-(x**2))
        # Proteção matemática para o arccos (domínio [-1, 1])
        if val > 1: val = 1
        return math.acos(val)
    except:
        return x # Retorna o próprio x em caso de erro matemático para não travar

# F2: x^3 - x - 1 = 0  =>  x = (x + 1)^(1/3)
def phi_2(x): 
    return (x + 1)**(1/3)

# F3: 4sin(x) - e^x = 0  =>  4sin(x) = e^x  =>  x = ln(4sin(x))
def phi_3(x):
    # Proteção: log só aceita positivo
    val = 4 * math.sin(x)
    if val <= 0: return x 
    return math.log(val)

# F4: x*log(x) - 1 = 0  =>  log(x) = 1/x  =>  x = e^(1/x)
def phi_4(x):
    if x == 0: return x
    return math.exp(1/x)


def ponto_fixo(phi, x0, precision, inter_max) -> tuple:
    """
    phi: Função de iteração x = phi(x)
    x0: Chute inicial
    """
    inicio = time.perf_counter()
    count = 0
    f_count = 0 # Contamos avaliações de phi(x)
    
    x_old = x0
    raiz = None
    
    while count < inter_max:
        try:
            # Passo principal
            x_new = phi(x_old)
            f_count += 1
            
            # Critério de Parada
            if abs(x_new - x_old) < precision:
                raiz = x_new
                break
            
            x_old = x_new
            count += 1
            
        except (ValueError, OverflowError):
            # O MPF tava falhando em alguns casos, isso evita o crash
            break

    # Cálculos finais
    tempo_total = time.perf_counter() - inicio
    tempo_medio = tempo_total / count if count > 0 else 0
    
    return raiz, count, f_count, tempo_total, tempo_medio


chutes_iniciais = [1.5, 1.5, 0.5, 1.5] # Escolhidos perto das raízes conhecidas
phis = [phi_1, phi_2, phi_3, phi_4]
epsilons = [1e-4, 1e-6, 1e-5, 1e-7]


for i in range(4):
    res = ponto_fixo(phis[i], chutes_iniciais[i], epsilons[i], 100)
    
    raiz, k, evals, t_total, t_medio = res
    
    print(f"\nFunção {i+1} (Phi específica):")
    if raiz is not None:
        print(f"  Raiz: {raiz:.8f}")
        print(f"  Iterações: {k}")
        print(f"  Avaliações de Phi: {evals}")
        print(f"  Tempo Total: {t_total:.6f}s")
    else:
        print("  ERRO: Não convergiu ou erro matemático na Phi.")

# tive que escrever muito comentários para explicar o código, vou dar o espaço anterior não #cansado kkkk


Função 1 (Phi específica):
  Raiz: 1.44745118
  Iterações: 6
  Avaliações de Phi: 7
  Tempo Total: 0.000012s

Função 2 (Phi específica):
  Raiz: 1.32471801
  Iterações: 8
  Avaliações de Phi: 9
  Tempo Total: 0.000004s

Função 3 (Phi específica):
  Raiz: 1.36495669
  Iterações: 10
  Avaliações de Phi: 11
  Tempo Total: 0.000006s

Função 4 (Phi específica):
  Raiz: 1.76322286
  Iterações: 28
  Avaliações de Phi: 29
  Tempo Total: 0.000005s


## Metodo de Newton-Raphson
O Método de Newton-Raphson é um dos métodos mais poderosos para determinação de raízes. Geometricamente, ele substitui a curva da função $f(x)$ por sua reta tangente no ponto atual $x_k$. A intersecção dessa tangente com o eixo das abscissas fornece a próxima aproximação $x_{k+1}$.
Matematicamente, ele é derivado da expansão em Série de Taylor truncada no termo linear. Partindo de uma estimativa inicial $x_0$, gera-se a sequência de aproximações através da fórmula iterativa:
$$x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}$$

In [39]:
# derivadas das funções originais
def dfuncao_1(x): 
    return -2*x*math.exp(-(x**2)) + math.sin(x) # Derivada de e^(-x^2) - cos(x)

def dfuncao_2(x): 
    return 3*(x**2) - 1 # Derivada de x^3 - x - 1

def dfuncao_3(x): 
    return 4*math.cos(x) - math.exp(x) # Derivada de 4sin(x) - e^x

def dfuncao_4(x): 
    return math.log(x) + 1 # Derivada de x*ln(x) - 1

def newton_raphson(f, df, x0, precision, inter_max) -> tuple:
    """
    f: Função f(x)
    df: Derivada f'(x)
    x0: Chute inicial
    """
    inicio = time.perf_counter()
    count = 0
    f_count = 0 # Contamos chamadas de f e df
    
    x = x0
    raiz = None
    
    while count < inter_max:
        # Avaliações
        fx = f(x)
        dfx = df(x)
        f_count += 2
        
        # Critério de Parada
        if abs(fx) < precision:
            raiz = x
            break
            
        # Proteção contra divisão por zero (Derivada nula)
        if dfx == 0:
            print("Erro: Derivada zero encontrada. Método falhou.")
            break
            
        # Passo de Newton: x_novo = x - f(x)/f'(x)
        x_novo = x - (fx / dfx)
        
        # Critério de Parada
        if abs(x_novo - x) < precision:
            raiz = x_novo
            break
            
        x = x_novo
        count += 1

    # Cálculos finais
    tempo_total = time.perf_counter() - inicio
    tempo_medio = tempo_total / count if count > 0 else 0
    
    return raiz, count, f_count, tempo_total, tempo_medio

# Execução Específica para Newton-Raphson
funcs = [funcao_1, funcao_2, funcao_3, funcao_4]
dfuncs = [dfuncao_1, dfuncao_2, dfuncao_3, dfuncao_4]
chutes = [1.5, 1.5, 0.5, 1.5] # x0
epsilons = [1e-4, 1e-6, 1e-5, 1e-7]

for i in range(4):
    res = newton_raphson(funcs[i], dfuncs[i], chutes[i], epsilons[i], 100)
    
    raiz, k, evals, t_total, t_medio = res
    
    print(f"\nFunção {i+1}:")
    if raiz is not None:
        print(f"  Raiz: {raiz:.8f}")
        print(f"  Iterações: {k}")
        print(f"  Avaliações (f + f'): {evals}")
        print(f"  Tempo Total: {t_total:.8f}s")
    else:
        print("  ERRO: Não convergiu.")


Função 1:
  Raiz: 1.44741635
  Iterações: 2
  Avaliações (f + f'): 6
  Tempo Total: 0.00001100s

Função 2:
  Raiz: 1.32471817
  Iterações: 3
  Avaliações (f + f'): 8
  Tempo Total: 0.00000510s

Função 3:
  Raiz: 0.37055808
  Iterações: 3
  Avaliações (f + f'): 8
  Tempo Total: 0.00000570s

Função 4:
  Raiz: 1.76322283
  Iterações: 3
  Avaliações (f + f'): 8
  Tempo Total: 0.00000300s


### Análise de Desempenho
* **Convergência:** Quadrática ($p=2$). Isso significa que, próximo da raiz, o número de casas decimais corretas dobra a cada iteração. É muito mais rápido que a Bissecção e a Posição Falsa.
* **Custo Computacional:** Alto por iteração. Exige calcular **duas** funções a cada passo: a função original $f(x)$ e sua derivada $f'(x)$.
* **Riscos:**
    * O método falha se $f'(x_k) \approx 0$ (tangente horizontal).
    * Pode divergir (oscilar) se o chute inicial $x_0$ estiver muito longe da raiz ou se a função tiver pontos de inflexão próximos.

## Metodo Secante
O Método da Secante surge como uma alternativa ao Método de Newton-Raphson para situações onde calcular a derivada $f'(x)$ é difícil ou custoso computacionalmente.
Em vez de usar a derivada exata (reta tangente), o método aproxima a inclinação da tangente usando uma **reta secante** que passa pelos dois pontos anteriores da iteração ($x_{k}$ e $x_{k-1}$). Diferente de Newton (que precisa de 1 ponto) e Bissecção (que precisa de um intervalo), a Secante precisa de **dois pontos iniciais** ($x_0$ e $x_1$) para começar a traçar a reta. A fórmula de recorrência é:

$$x_{k+1} = x_k - \frac{f(x_k) \cdot (x_k - x_{k-1})}{f(x_k) - f(x_{k-1})}$$

In [40]:
def funcao_1(x): return math.exp(-(x**2)) - math.cos(x)
def funcao_2(x): return x**3 - x - 1
def funcao_3(x): return 4*math.sin(x) - math.exp(x)
def funcao_4(x): return x*math.log(x) - 1

# --- Configurações ---
epsilons = [1e-4, 1e-6, 1e-5, 1e-7]

#intervalos
chutes = [
    (1, 2), # Func 1
    (1, 2), # Func 2
    (0, 1), # Func 3
    (1, 2)  # Func 4
]

def secante(f, x0, x1, precision, inter_max) -> tuple:
    """
    f: Função f(x)
    x0, x1: Pontos iniciais
    """
    inicio = time.perf_counter()
    count = 0
    f_count = 0
    
    # Avaliação inicial dos dois pontos
    fx0 = f(x0)
    fx1 = f(x1)
    f_count += 2
    
    raiz = None
    
    # Verificação se já começamos na raiz
    if abs(fx0) < precision: return x0, 0, f_count, (time.perf_counter() - inicio), 0
    if abs(fx1) < precision: return x1, 0, f_count, (time.perf_counter() - inicio), 0

    while count < inter_max:
        # Proteção contra divisão por zero
        if abs(fx1 - fx0) < 1e-15:
            print("Aviso: Divisão por zero iminente (f(x1) ~= f(x0)).")
            break

        x_new = x1 - (fx1 * (x1 - x0)) / (fx1 - fx0) # Fórmula da Secante >> atualizando novo ponto
        
        fx_new = f(x_new)
        f_count += 1
        
        # Critérios de Parada
        if abs(fx_new) < precision or abs(x_new - x1) < precision:
            raiz = x_new
            break
            
        # Atualização dos pontos
        x0 = x1
        fx0 = fx1
        
        x1 = x_new
        fx1 = fx_new
        
        count += 1

    # Cálculos finais
    tempo_total = time.perf_counter() - inicio
    tempo_medio = tempo_total / count if count > 0 else 0
    
    # não convergiu >> retorna o último calculado
    if raiz is None: raiz = x_new
    
    return raiz, count, f_count, tempo_total, tempo_medio


funcs = [funcao_1, funcao_2, funcao_3, funcao_4]


for i in range(4):
    x0_val = chutes[i][0]
    x1_val = chutes[i][1]
    
    res = secante(funcs[i], x0_val, x1_val, epsilons[i], 100)
    
    raiz, k, evals, t_total, t_medio = res
    
    print(f"\nFunção {i+1}:")
    print(f"  Raiz: {raiz:.8f}")
    print(f"  Iterações: {k}")
    print(f"  Avaliações: {evals}")
    print(f"  Tempo Total: {t_total:.8f}s")


Função 1:
  Raiz: 1.44741345
  Iterações: 4
  Avaliações: 7
  Tempo Total: 0.00001410s

Função 2:
  Raiz: 1.32471797
  Iterações: 5
  Avaliações: 8
  Tempo Total: 0.00000980s

Função 3:
  Raiz: 0.37055810
  Iterações: 6
  Avaliações: 9
  Tempo Total: 0.00000660s

Função 4:
  Raiz: 1.76322283
  Iterações: 3
  Avaliações: 6
  Tempo Total: 0.00000360s


### Análise de Desempenho
* **Convergência:** Superlinear (ordem $p \approx 1.618$). É mais rápido que a Bissecção e Posição Falsa, mas levemente mais lento que o Newton.
* **Vantagem:** Não exige cálculo de derivadas.
* **Custo:** Apenas 1 avaliação de função por iteração (pois $f(x_{k-1})$ já foi calculado no passo anterior).
* **Risco:** Como não verifica troca de sinal (não "cerca" a raiz), o método pode divergir se a função não for bem comportada ou os chutes iniciais estiverem ruins.